## Notes 
<b> 2/18/2022 <b>
* For SAR, only projects that have empty phase completion dates are included. 
* Projects with NO allocation amount won't appear in SAR. 
* Wants to compare cols from previous SAR with current SAR b/c she has to bold any changes in black for CTC.
* For PAP, problem is prior col is all lumped together for projects before 2020? Highlands sheet does not differentiate between the years. Wants each year to be separated out.
    
<b> To do </b>
* Make sure all the values in date columns are replaced properly. 
* Make sure grant recipients don't have multiple variations of the same recipient.
* Change col names to match the new SAR report sent over 2/15.
* SAR: allocation amount col isn't from highlands sheet, it's taken by adding GGRF Alloc + PTA-SB1. Add them back in, add the column...

In [1]:
import numpy as np
import pandas as pd
import TIRCP_functions
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/tircp/"
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:.2f}".format

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
#Load in Crosswalks 
FILE_NAME3 = "Allocation_PPNO_Crosswalk.csv"
allocation_ppno_crosswalk = pd.read_csv(f"{GCS_FILE_PATH}{FILE_NAME3}")
    
#Allocation PPNO Crosswalk
FILE_NAME4 = "Projects_PPNO.xlsx"
project_ppno_crosswalk = pd.read_excel(f"{GCS_FILE_PATH}{FILE_NAME4}")


### Loading in Scripts

In [3]:
test_SAR = TIRCP_functions.semi_annual_report()

In [4]:
test_program = TIRCP_functions.program_allocation_plan()

In [5]:
alloc_test = TIRCP_functions.allocation()

In [6]:
tableau_test = TIRCP_functions.tableau()

In [7]:
project_test = TIRCP_functions.project()

### Testing

In [13]:
### SHEET CONNECTED TO TABLEAU ### 
def tableau():
    #Keeping only the columns we want
    df = project_test #project() CHANGE BACK
    df = df[['Award_Year', 'Project_#','Local_Agency','Project_Title','PPNO', 'County', 
    'Key_Project_Elements','TIRCP_project_sheet','Allocated_Amount',
     'Expended_Amt_project_sheet']]
    
    #Getting percentages & filling in with 0
    df['Expended_Percent'] = df['Expended_Amt_project_sheet']/df['Allocated_Amount']
    df['Allocated_Percent'] = df['Allocated_Amount']/df['TIRCP_project_sheet']
    df[['Expended_Percent','Allocated_Percent']] = df[['Expended_Percent','Allocated_Percent']].fillna(value=0)
    
    #Categorizing expended percentage into bins
    def expended_percent(row):
            if row.Expended_Percent == 0:
                return "No expenditure recorded"
            elif ((row.Expended_Percent > 0) and (row.Expended_Percent < .50)):
                return "1-50"
            elif row.Expended_Percent < 0.71:
                return "51-70"
            else:
                return "71-100"
    df["Expended_Percent_Group"] = df.apply(lambda x: expended_percent(x), axis=1)
    
    # Categorize years and expended_percent_group into bins
    def progress(df):   
        if (df['Award_Year'] == 2015) and (df['Expended_Percent_Group'] == "1-50"):
            return 'Behind'
        elif (df['Award_Year'] == 2015) and (df['Expended_Percent_Group'] == "51-70"):
            return 'On Track'
        elif (df['Award_Year'] == 2015) and (df['Expended_Percent_Group'] == "71-100"):
            return 'On Track'
        elif (df['Award_Year'] == 2016) and (df['Expended_Percent_Group'] == "1-50"):
            return 'Behind'
        elif (df['Award_Year'] == 2016) and (df['Expended_Percent_Group'] == "71-100"):
            return 'On Track'
        elif (df['Award_Year'] == 2016) and (df['Expended_Percent_Group'] == "51-70"):
            return 'On Track'
        elif (df['Award_Year'] == 2018) and (df['Expended_Percent_Group'] == "1-50"):
            return 'On Track'
        elif (df['Award_Year'] == 2018) and (df['Expended_Percent_Group'] == "51-70"):
            return 'Ahead'
        elif (df['Award_Year'] == 2018) and (df['Expended_Percent_Group'] == "71-100"):
            return 'Ahead'
        elif (df['Award_Year'] == 2020) and (df['Expended_Percent_Group'] == "1-50"):
            return 'On Track'
        elif (df['Award_Year'] == 2020) and (df['Expended_Percent_Group'] == "51-70"):
            return 'Ahead'
        elif (df['Award_Year'] == 2020) and (df['Expended_Percent_Group'] == "71-100"):
            return 'Ahead'
        else: 
            return "No Expenditures"

    df['Progress'] = df.apply(progress, axis = 1)
    
    #Which projects are large,small, medium
    p75 = df.TIRCP_project_sheet.quantile(0.75).astype(float)
    p25 = df.TIRCP_project_sheet.quantile(0.25).astype(float)
    p50 = df.TIRCP_project_sheet.quantile(0.50).astype(float)
    
    def project_size (row):
        if ((row.TIRCP_project_sheet > 0) and (row.TIRCP_project_sheet < p25)):
            return "Small"
        elif ((row.TIRCP_project_sheet > p25) and (row.TIRCP_project_sheet < p75)):
            return "Medium"
        elif ((row.TIRCP_project_sheet > p50) and (row.TIRCP_project_sheet > p75 )):
            return "Large"
        else:
            return "$0 recorded for TIRCP"
        
    df["Project_Category"] = df.apply(lambda x: project_size(x), axis=1)
     #Rename cols to the right names
    df = (df.rename(columns = {'Expended_Amt_project_sheet':'Expended_Amount', 
                                                'TIRCP_project_sheet': "TIRCP_Amount"}
                  ))
    ### GCS ###
    #df.to_csv(f'{GCS_FILE_PATH}df_tableau_sheet.csv', index = False)
    return df 

In [15]:
tableau_test = tableau()
tableau_test

/tmp/ipykernel_1218/703673757.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_1218/703673757.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_1218/703673757.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_1218/70367

,Award_Year,Project_#,Local_Agency,Project_Title,PPNO,County,Key_Project_Elements,TIRCP_Amount,Allocated_Amount,Expended_Amount,Expended_Percent,Allocated_Percent,Expended_Percent_Group,Progress,Project_Category
0,2015,1,Antelope Valley Transit Authority (AVTA),Regional Transit Interconnectivity & Environme...,CP005,LA,Purchase 13 60-foot articulated BRT buses and ...,24403000.00,24403000,21714177.53,0.89,1.00,71-100,On Track,Medium
1,2015,2,Capitol Corridor Joint Powers Authority,Travel Time Reduction Project,CP012,VAR,Track and curve improvements between San Jose ...,4620000.00,4620000,4619999.90,1.00,1.00,71-100,On Track,Small
2,2015,3,Los Angeles County Metropolitan Transportation...,Willowbrook/Rosa Parks Station & Blue Line Lig...,CP015,LA,Replace Blue Line signal system; install new t...,38494000.00,38494000,38494000.00,1.00,1.00,71-100,On Track,Medium
3,2015,4,Los Angeles-San Diego-San Luis Obispo Rail Cor...,Pacific Surfliner Transit Transfer Program,CP007,VAR,LOSSAN and 12 transit agencies from San Luis O...,1675000.00,1675000,277840.47,0.17,1.00,1-50,Behind,Small
4,2015,5,Montery-Salinas Transit,Monterey Bay Operations and Maintenance Facili...,CP013,MON,Renovation and expansion of the Monterey maint...,10000000.00,10000000,0.00,0.00,1.00,No expenditure recorded,No Expenditures,Medium
5,2015,6,Orange County Transportation Authority (OCTA),Bravo! Route 560 Rapid Buses,CP004,ORA,Purchase five 40-foot CNG buses for BRT Route ...,2320000.00,2320000,0.00,0.00,1.00,No expenditure recorded,No Expenditures,Small
6,2015,7,Sacramento Regional Transit District (SacRT),Sacramento Regional Transit's Refurbishment of...,CP001,SAC,Refurbishment of Seven UTDC Light Rail Vehicle...,6427000.00,6427000,0.00,0.00,1.00,No expenditure recorded,No Expenditures,Small
7,2015,8,San Diego Association of Governments (SANDAG),South Bay Bus Rapid Transit,CP003,SD,Bus Rapid Transit Route between Downtown San D...,4000000.00,4000000,4000000.00,1.00,1.00,71-100,On Track,Small
8,2015,9,San Diego Metropolitan Transit System (MTS),San Diego Metropolitan Transit System Trolley ...,CP008,SD,"Courthouse Trolley Station (Design), Courthous...",31986000.00,31936000,28176000.00,0.88,1.00,71-100,On Track,Medium
9,2015,10,San Francisco Municipal Transportation Agency,SFMTA Light Rail Vehicle Fleet Expansion,CP006,SF,Purchase 8 Zero-emission Light Rail Vehicles f...,41181000.00,41181000,0.00,0.00,1.00,No expenditure recorded,No Expenditures,Large


### Once I get the final data, compare the PPNO # across the 2 sheets 

In [ ]:
PPNO_project = set(project_test.PPNO.unique().tolist())
PPNO_allocation = set(alloc_test.PPNO.unique().tolist())

In [ ]:
PPNO_project - PPNO_allocation #checking for differences